In [1]:
# when changing riders, do a factory reset and give the folder key inside file list
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1HdRVZ5uOwQW7XlpOyqT43YP94BsYfMv0' in parents and trashed = False and mimeType != 'application/vnd.google-apps.folder'"}).GetList() #import swd files

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

title: rider4_ride9_raw.csv, id: 1MEU4LoBvFwowNcRhdJ32WOzzNMMdbOxC
downloading to /root/data/rider4_ride9_raw.csv
title: rider4_ride8_raw.csv, id: 1TFh4oVK_8sY7TlWpR-bzImARBnZdqeO-
downloading to /root/data/rider4_ride8_raw.csv
title: rider4_ride7_raw.csv, id: 1ICWGr_CUIQk8aeDAZ-w7byabfKROe4yy
downloading to /root/data/rider4_ride7_raw.csv
title: rider4_ride6_raw.csv, id: 1fq4JE7NbxRxYIaw3P6UwNg-Prr97I3Rx
downloading to /root/data/rider4_ride6_raw.csv
title: rider4_ride5_raw.csv, id: 1ctam-j3FoX2Ik_QLG7qNZJKtlO5E7zvN
downloading to /root/data/rider4_ride5_raw.csv
title: rider4_ride4_raw.csv, id: 1DGDsrJTKiVRWA4B2uNm4Jmhlvgre4r3T
downloading to /root/data/rider4_ride4_raw.csv
title: rider4_ride3_raw.csv, id: 1mf7mduKtuFRCDCzeLg9V32Ci0qudUFGq
downloading to /root/data/rider4_ride3_raw.csv
title: rider4_ride2_raw.csv, id: 1vwGV4GpzSeu_RxMUVu6Aro-DVeCeGwq2
downloading to /root/data/rider4_ride2_raw.csv
title: rider4_ride1_raw.csv, id: 1LM6Ukc48cibw-7qTOACoQcBWYPo1vrud
downloading to /root/

In [2]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mdate
import numpy as np
import math
from datetime import datetime
from google.colab import files

import statsmodels.api as sm
from scipy import stats

#import fbprophet
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn import svm
from sklearn.kernel_approximation import Nystroem

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# this to be run only when all data


import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import glob 
import copy
#tf.compat.v1.enable_eager_execution() 

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#read csv all data, rename the old files with _old
#all_files = glob.glob("//root//data//*raw.csv")
#test_files = glob.glob("//root//data//*old.csv")

#read csv rider wise, rename the test file as test_ and keep the rider files as is
all_files = glob.glob("//root//data//rider*.csv")
test_files = glob.glob("//root//data//test*.csv") # rename a file for testing as test_ in root/data or root/data2

li=[]
for filename in all_files:
    print('importing...',filename)
    df1 = pd.read_csv(filename, index_col=None, header=0)
    print('before drop',len(df1))
    df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
    df1.replace(r"^\s*true$",1,inplace=True, regex=True)
    df1.replace(r"^\s*false$",0,inplace=True, regex=True)
    df1.dropna(axis=0,how='any',inplace=True)
    print('after drop',len(df1))
    li.append(df1)

df = pd.concat(li, axis=0, ignore_index=True)
total_trn = len(df)
print('total_train',len(df))

li1=[]
for filename in test_files:
  print('importing...',filename)
  df1=pd.read_csv(filename, index_col=None, header=0)
  #df1[['cadence','altitude_smooth','grade_smooth','velocity_smooth','watts']].str.strip()
  print('before drop1',len(df1))
  df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
  df1.replace(r"^\s*true$",1,inplace=True, regex=True)
  df1.replace(r"^\s*false$",0,inplace=True, regex=True)
  #df1.replace("",np.NaN,inplace=True)
  #print(df1['watts'].head())
  df1.dropna(axis=0,how='any',inplace=True)
  #df1.to_csv('new3182519785_dropped.csv')
  #files.download('new3182519785_dropped.csv')
  print('after drop1',len(df1))
  li1.append(df1)

test_df = pd.concat(li1, axis=0, ignore_index=True)
test_df_copy = copy.deepcopy(test_df)
#test_df=test_df.dropna()
total_tst = len(test_df)
print('total_test',total_tst)
#print(test_df['watts'].empty())
#test_df_filtered = test_df[test_df['watts'].str.strip()!=np.NaN]
print(test_df.tail())
#df1 = pd.read_csv('_5.csv')
#df=df.append(df1, ignore_index=True)


#df=df[['timestamp(s)', 'position_lat(semicircles)', 'position_long(semicircles)', 'gps_accuracy(m)', 'altitude(m)',
#       'grade(%)', 'distance(m)', 'cadence(rpm)', 'speed(m/s)', 'power(watts)', 'left_right_balance', 'temperature(C)', 'acceleration']]

print (test_df.isnull().any())
print(test_df.columns)

importing... //root//data/rider4_ride9_raw.csv
before drop 14124
after drop 14124
importing... //root//data/rider4_ride5_raw.csv
before drop 8649
after drop 8649
importing... //root//data/rider4_ride6_raw.csv
before drop 7195
after drop 7195
importing... //root//data/rider4_ride3_raw.csv
before drop 14204
after drop 14204
importing... //root//data/rider4_ride7_raw.csv
before drop 8203
after drop 8203
importing... //root//data/rider4_ride8_raw.csv
before drop 7284
after drop 7284
importing... //root//data/rider4_ride1_raw.csv
before drop 4245
after drop 4245
importing... //root//data/rider4_ride4_raw.csv
before drop 8939
after drop 8939
total_train 72843
importing... //root//data/test_rider4_ride2_raw.csv
before drop1 7402
after drop1 7402
total_test 7402
      Unnamed: 0      timestamp(s)  ...  accel_sensor(m2/s)  accel_gps(m2/s)
7397        7397  26-06-2019 13.37  ...                1.65        -0.285552
7398        7398  26-06-2019 13.37  ...               -1.85        -0.687410
7399

In [4]:


#total_train 9770,8458,12962,22035
#total_test 2210,1354,2814,4707
#total_train 29315,25385,38893,66121
#total_test 6631,4064,8444,14124
#total_train 2930,,,6608
#total_test 663,,,1412

#minimal and extended feature headers. if you normalize and then use the data, remove weight from features, because it is a column of constants and will break the model.

# for all files, make sure you have the same no. of cols in old and new and in same order

header_sensor_minimal_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','weight_bike_rider_80','watts']#'weight_bike_rider_80',
header_gps_minimal_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','watts']#'weight_bike_rider_80',

header_sensor_extended_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','wind_resistance_sensor(N)',
                              'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_gps_extended_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','wind_resistance_gps(N)',
                           'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_sensor_minimal_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','watts' ] #,'weight_bike_rider_80'
header_gps_minimal_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)', 'watts' ]#,'weight_bike_rider_80'

header_sensor_extended_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','wind_resistance_sensor(N)',
                              'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',
header_gps_extended_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)','wind_resistance_gps(N)',
                           'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',

features_considered = header_sensor_extended_old # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old

features = df[features_considered]

print(features.head())

test_features_considered = header_sensor_extended_old # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old


test_features=test_df[test_features_considered]
test_features.columns=features_considered 

print(test_features.tail())

frames = [features,test_features] 
features = pd.concat(frames)  
features[features_considered] = features[features_considered].astype(float) 

# below code to be uncommented on for all data

#index1 = total_trn - 7943  # switch on when all files - last fle of rider 3
#index2 = total_trn + 7195  # switch on when all files - first file of rider 4
#train1 = features.iloc[:index1,:] # switch on when all files
#test = features.iloc[index1:index2, :] # switch on when all files
#train2 = features.iloc[index2:,:]# switch on when all files
#frames2 = [train1,train2,test] # switch on when all files - used when all data
#features = pd.concat(frames2)  # switch on when all files - used when all data
#total_tst = len(test)
#print(total_tst)


print(features.head())
print(features.tail())
print('total dataset',len(features))


#print(features[~features.applymap(np.isreal).all(1)])
#plotting individual graphs for features
#features.plot(subplots=True)
#plt.legend(loc=1)
#plt.savefig('features_plot.png')
#files.download("features_plot.png") 

#plt.show()

   distance(m)  speed(m/s)  ...  altitude(m)  watts
0       129.34      10.212  ...         33.0  275.0
1       140.59      10.194  ...         33.0  269.0
2       150.87      10.331  ...         33.0  259.0
3       161.23      10.461  ...         33.0  250.0
4       171.60      10.592  ...         33.0  240.0

[5 rows x 9 columns]
      distance(m)  speed(m/s)  ...  altitude(m)  watts
7397     65127.96      11.644  ...         44.2    1.0
7398     65138.61      11.153  ...         43.8    0.0
7399     65151.15      10.702  ...         43.6    0.0
7400     65159.72       9.574  ...         43.6    0.0
7401     65170.19       8.372  ...         43.8    0.0

[5 rows x 9 columns]
   distance(m)  speed(m/s)  ...  altitude(m)  watts
0       129.34      10.212  ...         33.0  275.0
1       140.59      10.194  ...         33.0  269.0
2       150.87      10.331  ...         33.0  259.0
3       161.23      10.461  ...         33.0  250.0
4       171.60      10.592  ...         33.0  240.0

[

In [5]:
#runs at all times to get the results


#train_split = len(train1)+len(train2) # only when all files
train_split=total_trn #when rider wise

#data normalization

features=features.values.astype('float')  #turn on when u want to normalize ( do not use for GLM)
data_mean=features[:train_split].mean(axis=0)  #turn on when u want to normalize ( do not use for GLM)
data_std=features[:train_split].std(axis=0)  #turn on when u want to normalize ( do not use for GLM)

data_min=features[:train_split].min(axis=0)  #turn on when u want to normalize ( do not use for GLM)
print('data_mean', data_mean, 'data_std',data_std)  #turn on when u want to normalize ( do not use for GLM)
features=(features-data_mean)/data_std  #turn on when u want to normalize ( do not use for GLM)

features_train = features[:train_split] #used when data normalized
fetures_test = features[train_split:] #used when data normalized
X_train = features_train[:,:8].astype('float') #used when data normalized
#print(X_train[0])
X_test = fetures_test[:,:8].astype('float') #used when data normalized
y_train = features_train[:,-1].astype('float') #used when data normalized
y_test = fetures_test[:,-1].astype('float') #used when data normalized
y_test=(y_test*data_std[-1]+data_mean[-1])

#features_train = features[:train_split] #used when data is non-normalized
#fetures_test = features[train_split:] #used when data is non-normalized
#X_train = features_train.iloc[:,:8].values.astype('float') #used when data is non-normalized
#print(X_train[0])
#X_test = fetures_test.iloc[:,:8].values.astype('float') #used when data is non-normalized
#y_train = features_train.iloc[:,-1].values.astype('float') #used when data is non-normalized
#y_test = fetures_test.iloc[:,-1].values.astype('float') #used when data is non-normalized
#y_watts_calc = fetures_test.iloc[:,-2].values.astype('float')


#y_watts_calc=y_watts_calc*data_std[-2]+data_mean[-2]
#print(y_train[0])



#print(len(features_train),len(fetures_test))
y_pred_svr=[]
y_pred_glm=[]
y_pred_linear=[]
y_pred_lasso=[]
print('mean y_test',np.mean(y_test))

model=['liner','lasso','svr']#, 'GLM'] #remove GLM if data is normalized
for type in model:
  if type == 'svr':
    regressor=svm.SVR(kernel='rbf', C=1, gamma='auto', degree=5, epsilon=.1,coef0=1,  tol=1)
    '''
    regressor = linear_model.SGDRegressor()
    feature_map_nystroem = Nystroem(gamma = .2, random_state = 1, n_components = 2000)
    X_train_transformed = feature_map_nystroem.fit_transform(X_train)
    X_test_transformed = feature_map_nystroem.fit_transform(X_test)
    regressor.fit(X_train_transformed, y_train)
    y_pred = regressor.predict(X_test_transformed)
    '''
    regressor.fit(X_train,y_train)
    y_pred = regressor.predict(X_test)

    print(y_test)
    y_pred=y_pred*data_std[-1]+data_mean[-1]

    print('Root Mean Squared Error (SVR Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (SVR Regression):', metrics.mean_absolute_error(y_test,y_pred))

    print('R2 Score (SVR):',metrics.r2_score(y_test,y_pred))

    print('mean y_pred (svr)', np.mean((y_pred)))
    y_pred_svr=y_pred
    #print(len(y_pred))
    continue

  if type=='liner':
    regressor = LinearRegression()  
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    y_pred=y_pred*data_std[-1]+data_mean[-1]
    print('Root Mean Squared Error (Linear Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (linear Regression):', metrics.mean_absolute_error(y_test,y_pred))
    print('R2 Score (Linear):',metrics.r2_score(y_test,y_pred))
    print('mean y_pred (Linear)', np.mean((y_pred)))
    y_pred_linear=y_pred
    print(y_test)
    #LinR=pd.DataFrame({'Actual Test': y_test, 'Predicted Linear': y_pred })
    #LinR.to_csv('LinR.csv')
    #np.savetxt('RMSE_Linear.txt',  [np.sqrt(metrics.mean_squared_error(y_test, y_pred))])
    continue
  if type=='lasso':
    regressor=linear_model.Lasso(alpha=2/3, tol=0.1)
    regressor.fit(X_train, y_train)
    print(regressor.coef_)
    y_pred = regressor.predict(X_test)
    y_pred=y_pred*data_std[-1]+data_mean[-1]
    print('Root Mean Squared Error (Lasso Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (Lasso Regression):', metrics.mean_absolute_error(y_test,y_pred))
    print('R2 Score (Lasso):',metrics.r2_score(y_test,y_pred))
    print('mean y_pred (lasso)', np.mean((y_pred)))
    y_pred_lasso = y_pred
    print(y_test)
    #LS=pd.DataFrame({'Actual Test': y_test, 'Predicted Linear': y_pred })
    #LS.to_csv('LS.csv')
    #np.savetxt('RMSE_Lasso.txt',  [np.sqrt(metrics.mean_squared_error(y_test, y_pred))])
    continue

  if type == 'GLM': # GLM cannot be run with normalized power, because then dependent variable becomes negative
    exog, endog = sm.add_constant(X_train), y_train
    mod = sm.GLM(endog,exog,family=sm.families.Gaussian(link=sm.families.links.log()))
    res = mod.fit(maxiter=100)
    with np.errstate(divide='ignore'):
      yhat = res.predict(sm.add_constant(X_test))
    sqared_diff_sensor = 0
    for i in range(len(y_test)):
      sqared_diff_sensor = sqared_diff_sensor + (y_test[i] - yhat[i])**2

    RMSE_sensor = math.sqrt((sqared_diff_sensor)/len(y_test))
   
    #print(RMSE_sensor)
    print('Root Mean Squared Error (GLM):', metrics.mean_squared_error(y_test,yhat,squared=False))
    print('Mean Abs Error (GLM):', metrics.mean_absolute_error(y_test,yhat))
    print('R2 Score (GLM):',metrics.r2_score(y_test,yhat))
    print('mean y_pred (glm)', np.mean((yhat)))
    print(y_test)

    y_pred_glm = yhat


#d={'y_test':y_test,'watts_calc':y_watts_calc,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr,'y_pred_glm':y_pred_glm}
d={'y_test':y_test,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr,'y_pred_glm':y_pred_glm} # to be used with rider 4

pred_df=pd.DataFrame(d)
pred_df.to_csv('predict_r1_sensor_ext.csv')
files.download('predict_r1_sensor_ext.csv')



data_mean [4.36176613e+04 8.45730576e+00 6.62704919e-03 4.58272116e-01
 3.42328300e+01 1.58618346e+01 6.74903011e+01 6.30434932e+01
 1.79107169e+02] data_std [2.88180099e+04 2.28682417e+00 6.50529032e+00 3.17041106e+00
 3.05941256e+02 5.80394497e+00 2.24068943e+01 4.29250804e+01
 9.70915596e+01]
mean y_test 211.56334777607404
Root Mean Squared Error (Linear Regression): 64.41647167514742
Mean Abs Error (linear Regression): 48.17839609346608
R2 Score (Linear): 0.3405631379606737
mean y_pred (Linear) 188.3776136098382
[331. 331. 333. ...   0.   0.   0.]
[-0.  0.  0.  0.  0.  0.  0.  0.]
Root Mean Squared Error (Lasso Regression): 85.70802877699828
Mean Abs Error (Lasso Regression): 70.2149342465824
R2 Score (Lasso): -0.16740719934381
mean y_pred (lasso) 179.107168874797
[331. 331. 333. ...   0.   0.   0.]
[331. 331. 333. ...   0.   0.   0.]
Root Mean Squared Error (SVR Regression): 60.873892563095545
Mean Abs Error (SVR Regression): 44.19749540658242
R2 Score (SVR): 0.4111000727226416
me

ValueError: ignored

In [0]:
d={'y_test':y_test,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr} # to be used with rider 4

pred_df=pd.DataFrame(d)
pred_df.to_csv('predict_r1_sensor_ext.csv')
files.download('predict_r1_sensor_ext.csv')

In [5]:
#do not use this: this was version 1 implementation for rider wise


header_sensor_watts_new =[ 'cadence', 'temp', 'altitude', 'grade_smooth','distance',
                        'weight_bike_rider_70', 'speed_from_sensor(m/s)', 'wind_resistance_sensor(N)', 'accel_sensor(m2/s)', 'watts' ]

header_gps_watts_new = [ 'cadence',  'temp', 'altitude',   'grade_smooth', 'distance_calc(m)',
                        'weight_bike_rider_70',  'speed_from_gps(m/s)',  'wind_resistance_gps(N)',   'accel_gps(m2/s)',   'watts']

header_sensor_strava=[ 'cadence', 'temp', 'altitude', 'grade_smooth',  
        'weight_bike_rider_70', 'speed_from_sensor(m/s)', 'wind_resistance_sensor(N)', 'accel_sensor(m2/s)', 'watts_calc' ]

header_gps_strava=[   'cadence',  'temp', 'altitude',   'grade_smooth',
        'weight_bike_rider_70',  'speed_from_gps(m/s)',  'wind_resistance_gps(N)',   'accel_gps(m2/s)',   'watts_calc']

header_sensor_watts_all = ['moving', 'distance',  'heartrate', 'cadence','lat', 'lng', 'temp', 'altitude','weight_bike_rider_70',
                      'speed_from_sensor(m/s)','wind_resistance_sensor(N)','accel_sensor(m2/s)','watts']

header_sensor_minimal_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','watts_calc','watts']#,'weight_bike_rider_80'
header_gps_minimal_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','weight_bike_rider_80','watts_calc', 'watts']

header_sensor_extended_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','weight_bike_rider_80','wind_resistance_sensor(N)',
                              'temp','cadence','altitude','heartrate','watts_calc','watts']
header_gps_extended_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','weight_bike_rider_80','wind_resistance_gps(N)',
                           'temp','cadence','altitude','heartrate','watts_calc','watts']

new_files = glob.glob("//root//data//*.csv")
li1=[]
lastdf=0
for filename in new_files:
    print('Importing...',filename)
    df1=pd.read_csv(filename, index_col=None, header=0)
    #df1[['cadence','altitude_smooth','grade_smooth','velocity_smooth','watts']].str.strip()
    print('before drop',len(df1))
    df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
    df1.replace(r"^\s*true$",1,inplace=True, regex=True)
    df1.replace(r"^\s*false$",0,inplace=True, regex=True)
    df1.dropna(axis=0,how='any',inplace=True)
    print('after drop',len(df1))
    lastdf=len(df1)
    li1.append(df1)

new_df = pd.concat(li1, axis=0, ignore_index=True)
new_header = header_sensor_minimal_new
new_df_data = new_df[new_header]
features = new_df_data
features


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Importing... //root//data/rider1_ride2_raw.csv
before drop 13314
after drop 13306
Importing... //root//data/rider3_ride2_raw.csv
before drop 14263
after drop 14258
Importing... //root//data/rider1_ride4_raw.csv
before drop 6636
after drop 6631
Importing... //root//data/rider4_ride6_raw_old.csv
before drop 7195
after drop 7195
Importing... //root//data/rider1_ride1_raw.csv
before drop 14153
after drop 13697
Importing... //root//data/rider2_ride2_raw.csv
before drop 9572
after drop 9555
Importing... //root//data/rider3_ride3_raw.csv
before drop 12042
after drop 12041
Importing... //root//data/rider4_ride3_raw_old.csv
before drop 14204
after drop 14204
Importing... //root//data/rider4_ride7_raw_old.csv
before drop 8203
after drop 8203
Importing... //root//data/rider4_ride1_raw_old.csv
before drop 4245
after drop 4245
Importing... //root//data/rider3_ride5_raw.csv
before drop 8447
after drop 8444
Importing... //root//data/rider3_ride1_raw.csv
before drop 4651
after drop 4651
Importing... /

,distance,speed_from_sensor(m/s),accel_sensor(m2/s),grade_smooth,watts_calc,watts
0,9.3,6.2,4.2,1.2,329.0,176
1,14.9,5.6,-0.6,0.0,8.0,201
2,17.8,2.9,-2.7,0.0,0.0,190
3,24.0,6.2,3.3,0.0,271.0,181
4,30.3,6.3,0.1,0.0,68.0,198
...,...,...,...,...,...,...
192972,63494.3,10.4,-0.7,-10.8,0.0,0
192973,63503.6,9.3,-1.1,-11.7,0.0,0
192974,63511.6,8.0,-1.3,-13.7,0.0,0
192975,63518.0,6.4,-1.6,-15.1,0.0,0
